## Summary

Note book này được sao chép và chỉnh sửa lại từ project của [Nick Kuzmenkov](https://www.kaggle.com/nickuzmenkov)
* Preprocessing: loại bỏ 77 ảnh bị bị trùng lặp, chuyễn nhãn thành dạng one hot vector, tạo augmentation cho tập dữ liệu và chia tập dữ liệu thành 5 fold.
* Backbone: EfficientNetB4, EfficientNetB7, `noisy-student` weight.
* Optimizer: Adam, learning rate of 1e-3, ReduceLROnPlateau
* Image size: 600x600
* Augmentations: heavy augmentations with `albumentations` library.

Số điểm cao nhất đạt được: 0.837.

### Các notebook khác của nhóm:
1. [Revealing Duplicates notebook](https://www.kaggle.com/nvlinhh/int3414-22-n11-revealing-duplicate)
2. [Preprocessing notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-preprocessing)
3. [Training notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-training)

### Imports

In [ ]:
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.notebook import tqdm
import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import os

### Configurations
Việc chạy với 'mixed_float16' có nghĩa là lưu tensor ở dạng 32bit nhưng các phép tính được thực hiện ở 16 bit, cách này khiến cải thiện hiệu xuất.

Xem thêm chi tiết tại **[TensorFlow Documentation](https://www.tensorflow.org/guide/mixed_precision)**.

In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')



class CFG:
    
    '''
    keep these
    '''
    strategy = tf.distribute.get_strategy()
    batch_size = 16 * strategy.num_replicas_in_sync
    
    img_size = 600
    
    classes = np.array([
        'complex', 
        'frog_eye_leaf_spot', 
        'powdery_mildew', 
        'rust', 
        'scab'])
    root = '../input/plant-pathology-2021-fgvc8/test_images'
    
    '''
    tweak these
    '''
    seed = 42 # random seed we use for each operation
#     tta_steps = 1 # number of TTA folds, run without TTA if 0

### Helper functions

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.reshape(image, [CFG.img_size, CFG.img_size, 3])
    image = tf.cast(image, tf.float32) / 255.#chuyển sang dạng này do tpu chỉ hộ trợ float32 
    return image


def data_augment(image, label):
    image = tf.image.random_flip_left_right(image, seed=CFG.seed)
    image = tf.image.random_flip_up_down(image, seed=CFG.seed)
    
    k = tf.tf.random.uniform([], minval=0, maxval=4, dtype=tf.int64, seed=CFG.seed)
    image = tf.image.rot90(image, k=k)
    
    image = tf.image.random_hue(image, .1, seed=CFG.seed)
    image = tf.image.random_saturation(image, .8, 1.2, seed=CFG.seed)
    image = tf.image.random_contrast(image, .8, 1.2, seed=CFG.seed)
    image = tf.image.random_brightness(image, .1, seed=CFG.seed)
    
    return image, label

#config lại dạng image
feature_map = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_name': tf.io.FixedLenFeature([], tf.string)}

#đọc fild tfrec
def read_tfrecord(example):
    example = tf.io.parse_single_example(example, feature_map)
    image = decode_image(example['image'])
    label = example['image_name']
    return image, label


def get_dataset(filenames, ordered=True, shuffled=False, repeated=False, 
                augmented=False, cached=False, distributed=False):
    auto = tf.data.experimental.AUTOTUNE#hàm tối ưu
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=auto)
    if not ordered:
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic = False
        dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=auto)
    if shuffled:#xáo trộn
        dataset = dataset.shuffle(2048, seed=CFG.seed)
    if repeated:#repeat ảnh
        dataset = dataset.repeat()
    dataset = dataset.batch(CFG.batch_size)
    if augmented:#tạo augumentation
        dataset = dataset.map(data_augment, num_parallel_calls=auto)
    if cached:#lưu bộ nhớ cached
        dataset = dataset.cache()
    dataset = dataset.prefetch(auto)
    if distributed:#phân phối data set trên nhiều tpu
        dataset = CFG.strategy.experimental_distribute_dataset(dataset)
    return dataset


def get_model():
    model = tf.keras.models.Sequential(name='EfficientNetB7')
    
    model.add(efn.EfficientNetB7(
        include_top=False,
        input_shape=(CFG.img_size, CFG.img_size, 3),
        weights=None,
        pooling='avg'))
    
    model.add(tf.keras.layers.Dense(len(CFG.classes), 
        kernel_initializer=tf.keras.initializers.RandomUniform(seed=CFG.seed),
        bias_initializer=tf.keras.initializers.Zeros(), name='dense_top'))
    model.add(tf.keras.layers.Activation('sigmoid', dtype='float32'))
    
    return model

In [ ]:
def _serialize_image(path):
    image = tf.io.read_file(path)
    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    image = tf.cast(image, tf.uint8)
    return tf.image.encode_jpeg(image).numpy()


def _serialize_sample(image, name):
    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'image_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[name]))}
    sample = tf.train.Example(features=tf.train.Features(feature=feature))
    return sample.SerializeToString()

#lưu tập test du
def serialize_test():
    samples = []
    
    for path in os.listdir(CFG.root):
        image = _serialize_image(os.path.join(CFG.root, path))
        name = path.encode()
        samples.append(_serialize_sample(image, name))
    
    with tf.io.TFRecordWriter('test.tfrec') as writer:
        [writer.write(x) for x in tqdm(samples, total=len(samples))]

### Test images serialization
.

In [ ]:
serialize_test()

### Inspect test images 

In [ ]:
paths = os.listdir(CFG.root)[:3]

figure, axes = plt.subplots(1, 3, figsize=[20, 10])

for i, path in enumerate(paths):
    image = plt.imread(os.path.join(CFG.root, path))

    axes[i].imshow(image)
    axes[i].axis('off')
    
plt.show()

## Run prediction
Chạy mô hình với 3 ảnh của tập test được cho

In [ ]:
size = len(os.listdir(CFG.root)) #=3

filenames = tf.io.gfile.glob('*.tfrec')

 #tạo augmentation cho tập test
# if CFG.tta_steps > 0:
#     dataset = get_dataset(filenames, repeated=True, augmented=False)
# else:
dataset = get_dataset(filenames)   
    
predicts = np.zeros((size, len(CFG.classes)))#ma trận 3xCFG.classes, 
paths = tf.io.gfile.glob('../input/pp2021-tpu-tf-training/*.h5')

for path in tqdm(paths, total=len(paths)):

    with CFG.strategy.scope():
        model = get_model()
        model.load_weights(path)
   
    #chạy cho tập test có augmentation và tính điểm  trug bình
#     if CFG.tta_steps > 0:
#         steps = CFG.tta_steps * (size / CFG.batch_size + 1)

#         predict = model.predict(dataset, steps=steps)[:size * CFG.tta_steps] / len(paths)
#         predicts += np.mean(
#             predict.reshape(size, CFG.tta_steps, len(CFG.classes), order='F'), axis=1)

#     else:
predicts += model.predict(dataset) / len(paths)

## Threshold configuration
Tính threshold phù hợp nhất cho từng loại bệnh.

In [ ]:
df_true = pd.read_csv(
    '../input/pp2021-kfold-tfrecords-0/train.csv', index_col='image').drop('healthy', axis=1)
df_pred = pd.read_csv(
    '../input/pp2021-tpu-tf-training/oof_predicts.csv', index_col='image')

df_true = df_true.reindex(df_pred.index)#sắp xếp lại theo vị trí của df_pred
# print(df_pred.index)
# df_true.shape, df_pred.shape ((18549, 5), (18549, 5))

y_true = df_true.values
y_pred = df_pred.values
# print(y_true,y_pred)

'''
run evaluation for each threshold in [0, 1)
'''
thresholds = np.arange(.01, 1., .01)
# print(thresholds) [0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
#                     0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
#                     0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
#                     0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
#                     0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
#                     0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
#                     0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
#                     0.99]
scores = []

for threshold in thresholds:
    metric = tfa.metrics.F1Score(
        num_classes=len(CFG.classes), 
        average=None, 
        threshold=threshold)#lớn hơn threshold -> 1, nhỏ hơn threshold -> 0
    metric.update_state(y_true, y_pred)
#     print(y_pred)
#     print(metric.result().numpy())
    scores.append(metric.result().numpy())
#     break
#     print(scores)
# print(scores)
    
df = pd.DataFrame(columns=CFG.classes, data=scores, index=pd.Index(thresholds, name='threshold'))
df
'''
find maximum value for each class and the corresponding threshold
'''
thresholds = []#lưu threshold mà class có giá trị lớn nhất
scores = []#lưu giá trị f1 lớn nhất của classs

for x in CFG.classes:
    thresholds.append(df[x].idxmax())#tìm threshold mà x có giá trị lớn nhất
    scores.append(df[x].max())#tìm scores có giá tị lớn nhất
    print(f'{x}: {df.loc[.5, x]:.4f} >>> {df.loc[thresholds[-1], x]:.4f} ({thresholds[-1]:.2f})')#lấy threshold 0.5 để so sánh giữa khách quan nhất với lớn nhất
# print(df.loc[0.39])
# print(df.complex.sort_values())
# df.loc[thresholds[-1]]
print(f'\nmean score: {df.loc[.5].mean():.4f} >>> {np.mean(scores):.4f}')

## Create submission.csv


In [ ]:
# predicts1 = predicts.copy()
# print(predicts1)
# print(thresholds )
# for i in range(len(predicts1)):
#     predicts1[i] = predicts1[i] > thresholds #nếu > threshold thì bằng 1
#     print(predicts1[i])

for i in range(len(predicts)):
    predicts[i] = predicts[i] > thresholds #nếu > threshold thì bằng 1
    print(predicts[i])

    
predicts = predicts.astype('bool')
labels = []

#1 class nếu true thì chuyển sang dang string
for i in range(len(predicts)):
    labels.append(' '.join(CFG.classes[predicts[i]]))
# print(labels) ['complex frog_eye_leaf_spot scab', 'frog_eye_leaf_spot', 'scab']            
    
labels = ['healthy' if ('healthy' in x or x == '') else x for x in labels] #nếu xuất hiện nhãn heathy hay các nhãn trống thì gán nhãn healthy
  

df = pd.DataFrame({
    'image': os.listdir(CFG.root),
    'labels': labels})
# print(df)                   image                           labels
#                 0  ad8770db05586b59.jpg  complex frog_eye_leaf_spot scab
#                 1  c7b03e718489f3ca.jpg               frog_eye_leaf_spot
#                 2  85f8cb619c66b863.jpg                             scab

df.to_csv('submission.csv', index=False)
display(df.head())